## 카카오 로컬 API 활용
- 도로명 주소 ---> 위도, 경도 좌표

In [16]:
import requests, json
import pandas as pd
from urllib.parse import quote

- API Key

In [17]:
with open('keys/카카오apikey.txt') as file:
    kakao_key = file.read()

- URL 만들기

In [18]:
base_url = 'https://dapi.kakao.com/v2/local/search/address.json'
addr = '서울특별시 영등포구 경인로 지하843 (영등포동3가)'
url = f'{base_url}?query={quote(addr)}'

In [19]:
# Header: Authorization: KakaoAK ${REST_API_KEY}
header = {'Authorization': f'KakaoAK {kakao_key}'}

- Kakao Local API 호출하여 결과 가져오기

In [20]:
result = requests.get(url, headers=header).json()
result

{'documents': [{'address': {'address_name': '서울 영등포구 영등포동3가 33',
    'b_code': '1156010400',
    'h_code': '1156053500',
    'main_address_no': '33',
    'mountain_yn': 'N',
    'region_1depth_name': '서울',
    'region_2depth_name': '영등포구',
    'region_3depth_h_name': '영등포동',
    'region_3depth_name': '영등포동3가',
    'sub_address_no': '',
    'x': '126.906658750671',
    'y': '37.5164625580995'},
   'address_name': '서울 영등포구 경인로 지하 843',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '서울 영등포구 경인로 지하 843',
    'building_name': '영등포역지하도상가',
    'main_building_no': '843',
    'region_1depth_name': '서울',
    'region_2depth_name': '영등포구',
    'region_3depth_name': '영등포동3가',
    'road_name': '경인로',
    'sub_building_no': '',
    'underground_yn': 'Y',
    'x': '126.906658750671',
    'y': '37.5164625580995',
    'zone_no': '07305'},
   'x': '126.906658750671',
   'y': '37.5164625580995'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [21]:
result['documents'][0].keys()

dict_keys(['address', 'address_name', 'address_type', 'road_address', 'x', 'y'])

In [22]:
lat = float(result['documents'][0]['y'])
lng = float(result['documents'][0]['x'])
lat, lng

(37.5164625580995, 126.906658750671)

- 영등포소재 지하철역 정보 완성하기

In [23]:
df = pd.read_csv('data/영등포소재 지하철역.csv')
df

,이름,주소
0,영등포역,서울특별시 영등포구 경인로 지하843 (영등포동3가)
1,신도림역,서울특별시 구로구 경인로 625 (신도림동)
2,당산역,서울특별시 영등포구 당산로 229 (당산동6가)
3,여의도역,서울특별시 영등포구 여의나루로 지하40 (여의도동)
4,영등포구청역,서울특별시 영등포구 당산로 지하121 (당산동3가)


In [24]:
lat_list, lng_list = [], []
for i in df.index:
    url = f'{base_url}?query={quote(df["주소"][i])}'
    result = requests.get(url, headers=header).json()
    lat_list.append(float(result['documents'][0]['y']))
    lng_list.append(float(result['documents'][0]['x']))

In [25]:
df['위도'] = lat_list
df['경도'] = lng_list
df

,이름,주소,위도,경도
0,영등포역,서울특별시 영등포구 경인로 지하843 (영등포동3가),37.516463,126.906659
1,신도림역,서울특별시 구로구 경인로 625 (신도림동),37.507084,126.884719
2,당산역,서울특별시 영등포구 당산로 229 (당산동6가),37.534957,126.902731
3,여의도역,서울특별시 영등포구 여의나루로 지하40 (여의도동),37.521842,126.924485
4,영등포구청역,서울특별시 영등포구 당산로 지하121 (당산동3가),37.525700,126.896627


In [26]:
df.to_csv('data/영등포소재 지하철역2.csv', index=False)
pd.read_csv('data/영등포소재 지하철역2.csv')

,이름,주소,위도,경도
0,영등포역,서울특별시 영등포구 경인로 지하843 (영등포동3가),37.516463,126.906659
1,신도림역,서울특별시 구로구 경인로 625 (신도림동),37.507084,126.884719
2,당산역,서울특별시 영등포구 당산로 229 (당산동6가),37.534957,126.902731
3,여의도역,서울특별시 영등포구 여의나루로 지하40 (여의도동),37.521842,126.924485
4,영등포구청역,서울특별시 영등포구 당산로 지하121 (당산동3가),37.525700,126.896627


In [31]:
df.경도[0], df.위도[0]

(126.906658750671, 37.5164625580995)

In [41]:
name, W, K = df.이름[0], df.위도[0], df.경도[0]
name2, W2, K2 = df.이름[1], df.위도[1], df.경도[1]

url1 = f'https://map.kakao.com/link/to/{name},{W},{K}'
url2 = f'https://map.kakao.com/link//{name2},{W2},{K2}'
url, url2

('https://map.kakao.com/link/to/영등포역,37.5164625580995,126.906658750671',
 'https://map.kakao.com/link/as/ 신도림역,37.5070842748567,126.884718686628')

In [49]:
import requests

def get_directions(start, end):
    url = 'https://dapi.kakao.com/v2/local/geo/transcoord.json?'
    headers = {
        'Authorization': 'd28b241dedf87cfabcdefc6085cb4ed5'  # 여기에 본인의 REST API 키를 입력하세요
    }

    # 출발지 좌표 변환 (WGS84 좌표계를 카카오 좌표계로 변환)
    start_url = url + f'x={start[0]}&y={start[1]}&input_coord=WGS84&output_coord=WTM'
    start_response = requests.get(start_url, headers=headers)
    start_json = start_response.json()
    start_x = start_json['x']
    start_y = start_json['y']

    # 도착지 좌표 변환 (WGS84 좌표계를 카카오 좌표계로 변환)
    end_url = url + f'x={end[0]}&y={end[1]}&input_coord=WGS84&output_coord=WTM'
    end_response = requests.get(end_url, headers=headers)
    end_json = end_response.json()
    end_x = end_json['x']
    end_y = end_json['y']

    # 경로 탐색
    
    route_url = f'https://dapi.kakao.com/v2/local/path.json?startX={start_x}&startY={start_y}&endX={end_x}&endY={end_y}&option=trafast'
    route_response = requests.get(route_url, headers=headers)
    route_json = route_response.json()

    # 경로 정보 및 거리 출력
    if 'code' in route_json and route_json['code'] == -2:
        print("경로를 찾을 수 없습니다.")
    else:
        distance = route_json['route']['distance']
        print(f"출발지에서 도착지까지의 거리는 {distance}m 입니다.")
        print("경로 정보:")
        for path in route_json['route']['trafast']:
            print(path)

# 출발지와 도착지 좌표 설정 (위도, 경도)
start_point = (37.5662952, 126.9779451)  # 예시: 서울 시청
end_point = (37.566681, 126.978437)  # 예시: 덕수궁

get_directions(start_point, end_point)


KeyError: 'x'

In [48]:
import requests

def get_directions(start, end):
    url = 'https://dapi.kakao.com/v2/local/geo/transcoord.json?'
    headers = {
        'Authorization': 'd28b241dedf87cfabcdefc6085cb4ed5'  # 여기에 본인의 REST API 키를 입력하세요
    }

    # 출발지 좌표 변환 (WGS84 좌표계를 카카오 좌표계로 변환)
    start_url = url + f'x={start[0]}&y={start[1]}&input_coord=WGS84&output_coord=WTM'
    start_response = requests.get(start_url, headers=headers)
    start_json = start_response.json()
    start_x = start_json['x']
    start_y = start_json['y']

    # 도착지 좌표 변환 (WGS84 좌표계를 카카오 좌표계로 변환)
    end_url = url + f'x={end[0]}&y={end[1]}&input_coord=WGS84&output_coord=WTM'
    end_response = requests.get(end_url, headers=headers)
    end_json = end_response.json()
    end_x = end_json['x']
    end_y = end_json['y']

    # 경로 탐색
    
    route_url = f'kakaomap://route?sp={start_x},{start_y}&ep={end_x},{end_y}&by=CAR'
    route_response = requests.get(route_url, headers=headers)
    route_json = route_response.json()

    # 경로 정보 및 거리 출력
    if 'code' in route_json and route_json['code'] == -2:
        print("경로를 찾을 수 없습니다.")
    else:
        distance = route_json['route']['distance']
        print(f"출발지에서 도착지까지의 거리는 {distance}m 입니다.")
        print("경로 정보:")
        for path in route_json['route']['trafast']:
            print(path)

# 출발지와 도착지 좌표 설정 (위도, 경도)
start_point = (37.5662952, 126.9779451)  # 예시: 서울 시청
end_point = (37.566681, 126.978437)  # 예시: 덕수궁

get_directions(start_point, end_point)


KeyError: 'x'

In [53]:
import requests

def get_route_distance(start_x, start_y, end_x, end_y):
    base_url = 'https://dapi.kakao.com/v2/local/route'
    
    headers = {
        'Authorization': 'd28b241dedf87cfabcdefc6085cb4ed5'  # 여기에 본인의 REST API 키를 입력하세요
    }

    params = {
        'start': f"{start_y},{start_x}",
        'goal': f"{end_y},{end_x}",
        'option': 'trafast',  # 빠른 길 찾기 옵션 선택
        'vehicle_type': '1'   # 1은 자동차, 2는 자전거, 3은 도보
    }

    response = requests.get(base_url, headers=headers, params=params)
    json_data = response.json()

    if 'documents' in json_data:
        distance = json_data['documents'][0]['distance']
        print(f"출발지에서 도착지까지의 거리는 {distance}m 입니다.")
    else:
        print("거리를 계산할 수 없습니다.")

# 출발지와 도착지의 경도와 위도 설정
start_longitude, start_latitude = 37.5662952, 126.9779451
end_longitude, end_latitude = 37.566681, 126.978437
get_route_distance(start_longitude, start_latitude, end_longitude, end_latitude)


거리를 계산할 수 없습니다.
